In [2]:
#importing dependencies
import pandas as pd
import numpy as np
import hashlib
from pillowtalk import RedshiftStorageDB, GoogleSheetsWorkbook
from pillowtalk import RedshiftComputeDB

/Users/kelly.rice/opt/anaconda3/envs/pillowtalk_env/lib/python3.7/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# reading CTR conversions doc 
ctr_df = pd.read_csv('CTR_conversions.csv', index_col=False, encoding="ISO-8859-1")
ctr_df.head()

,ï»¿,ConversionDate,Md5CUS,OrderNumberCUS
0,0,2023-03-19,e1a5766ee1d273e73f4a943a0d12a35c,R545305888
1,1,2023-03-15,a32acee976888100e74dcda8f409fee2,C00470349
2,2,2023-03-16,d634ad53f3838117593ba4793adde7c1,C00470881
3,3,2023-01-04,1a22bb418515935568e30623790b0ecd,R503929753
4,4,2023-01-04,1a22bb418515935568e30623790b0ecd,R503929753


In [3]:
# cleaning up
ctr_df.drop(columns=ctr_df.columns[0], axis=1,  inplace=True)
ctr_df.head()

,ConversionDate,Md5CUS,OrderNumberCUS
0,2023-03-19,e1a5766ee1d273e73f4a943a0d12a35c,R545305888
1,2023-03-15,a32acee976888100e74dcda8f409fee2,C00470349
2,2023-03-16,d634ad53f3838117593ba4793adde7c1,C00470881
3,2023-01-04,1a22bb418515935568e30623790b0ecd,R503929753
4,2023-01-04,1a22bb418515935568e30623790b0ecd,R503929753


In [4]:
# checking data types 
ctr_df.dtypes

ConversionDate    object
Md5CUS            object
OrderNumberCUS    object
dtype: object

In [5]:
# converting date column to datetime
ctr_df["ConversionDate"] = pd.to_datetime(ctr_df["ConversionDate"])

In [6]:
# checking data types 
ctr_df.dtypes

ConversionDate    datetime64[ns]
Md5CUS                    object
OrderNumberCUS            object
dtype: object

In [7]:
# checking df shape
ctr_df.shape

(268, 3)

In [8]:
# reading in TRT conversions doc
trt_df = pd.read_csv('TRT_conversions.csv', index_col=False, encoding="ISO-8859-1")


In [9]:
# viewing
trt_df.head()

,ï»¿,ConversionDate,Md5CUS,OrderNumberCUS
0,0,2023-03-18,12ebd8f03227f75b34380ccf36935a26,R613121082
1,1,2023-03-18,12ebd8f03227f75b34380ccf36935a26,R613121082
2,2,2023-02-13,245ef5a850fdd154b90af7b2eefea0e3,C00455768
3,3,2023-01-27,27dc3c024600dfcd0cf7bc48baefe62c,R588323820
4,4,2023-02-03,27dc3c024600dfcd0cf7bc48baefe62c,C00451196


In [10]:
# cleaning up DF 
trt_df.drop(columns= trt_df.columns[0], axis=1,  inplace=True)
trt_df["ConversionDate"] = pd.to_datetime(trt_df["ConversionDate"])

In [11]:
# viewing
trt_df.head()

,ConversionDate,Md5CUS,OrderNumberCUS
0,2023-03-18,12ebd8f03227f75b34380ccf36935a26,R613121082
1,2023-03-18,12ebd8f03227f75b34380ccf36935a26,R613121082
2,2023-02-13,245ef5a850fdd154b90af7b2eefea0e3,C00455768
3,2023-01-27,27dc3c024600dfcd0cf7bc48baefe62c,R588323820
4,2023-02-03,27dc3c024600dfcd0cf7bc48baefe62c,C00451196


In [12]:
#checking data types 
trt_df.dtypes

ConversionDate    datetime64[ns]
Md5CUS                    object
OrderNumberCUS            object
dtype: object

In [13]:
# checking df shape 
trt_df.shape

(10383, 3)

In [5]:
# setting query from our Database
query = '''
select distinct order_number, order_completed_at, email
from elsa.f_orders_union as orders
left join elsa.dim_store_union as stores
on orders.store_code = stores.store_code
where
    not coalesce(orders.is_free_order , FALSE)
    and stores.store_category in ('US-Ecom','US-Retail')
    and orders.is_order_complete
    and order_completed_at >= '2023-01-01'
    and order_completed_at < '2023-04-01'
'''

In [6]:
# connecting to Redshift
with RedshiftComputeDB(prod=True) as rs:
    casperdb_customers = rs.query_to_df(query)

casperdb_customers.head()

2023-06-08 16:38:05,173 Found credentials in shared credentials file: ~/.aws/credentials
2023-06-08 16:38:05,810 Connected to RedshiftDB(host='compute', autocommit=False, prod=True).
2023-06-08 16:38:07,463 Committing transaction from context manager.
2023-06-08 16:38:07,491 Closed connection to RedshiftDB(host='compute', autocommit=False, prod=True).


,order_number,order_completed_at,email
0,C00448004,2023-01-27 19:19:05,d5a94eabaaee6f561917dfa9b1232cce
1,C00448227,2023-01-28 03:09:02,4ae2d8d5bd4eb21e0da70e46f3850778
2,C00456634,2023-02-15 19:30:37,694fe133f2a42436a3325a6a1da11ded
3,C00458370,2023-02-18 23:07:39,441f6851af162bb03c70439561955f65
4,C00458759,2023-02-19 16:28:17,45cb363900ac103e9113538a94c00a9f


In [7]:
# checking datatypes from casper DB
casperdb_customers.dtypes

order_number                  object
order_completed_at    datetime64[ns]
email                         object
dtype: object

In [9]:
#checking DF shape 
casperdb_customers.shape

(54770, 3)

In [18]:
# merge CTR dataframes 
ctr_customers_merged = ctr_df.merge(casperdb_customers, left_on = ['OrderNumberCUS','Md5CUS',], right_on = ['order_number','email'], how='left')





In [19]:
# viewing merged DF
ctr_customers_merged.head()

,ConversionDate,Md5CUS,OrderNumberCUS,order_number,order_completed_at,email
0,2023-03-19,e1a5766ee1d273e73f4a943a0d12a35c,R545305888,R545305888,2023-03-19 20:55:06.591,e1a5766ee1d273e73f4a943a0d12a35c
1,2023-03-15,a32acee976888100e74dcda8f409fee2,C00470349,C00470349,2023-03-15 11:59:41.000,a32acee976888100e74dcda8f409fee2
2,2023-03-16,d634ad53f3838117593ba4793adde7c1,C00470881,C00470881,2023-03-16 06:14:22.000,d634ad53f3838117593ba4793adde7c1
3,2023-01-04,1a22bb418515935568e30623790b0ecd,R503929753,R503929753,2023-01-04 19:00:58.000,1a22bb418515935568e30623790b0ecd
4,2023-01-04,1a22bb418515935568e30623790b0ecd,R503929753,R503929753,2023-01-04 19:00:58.000,1a22bb418515935568e30623790b0ecd


In [20]:
# checking for nulls
sum(ctr_customers_merged['email'].isna())


0

In [21]:
# exporting CSV
ctr_customers_merged.to_csv('CTR_conversion_matched.csv')

In [22]:
# merge TRT dataframes 

trt_customers_merged = trt_df.merge(casperdb_customers, left_on = ['OrderNumberCUS','Md5CUS',], right_on = ['order_number','email'], how='left')


In [23]:
# checking for nulls

sum(trt_customers_merged['email'].isna())


4

In [24]:
#viewing
ctr_customers_merged.head(20)

,ConversionDate,Md5CUS,OrderNumberCUS,order_number,order_completed_at,email
0,2023-03-19,e1a5766ee1d273e73f4a943a0d12a35c,R545305888,R545305888,2023-03-19 20:55:06.591,e1a5766ee1d273e73f4a943a0d12a35c
1,2023-03-15,a32acee976888100e74dcda8f409fee2,C00470349,C00470349,2023-03-15 11:59:41.000,a32acee976888100e74dcda8f409fee2
2,2023-03-16,d634ad53f3838117593ba4793adde7c1,C00470881,C00470881,2023-03-16 06:14:22.000,d634ad53f3838117593ba4793adde7c1
3,2023-01-04,1a22bb418515935568e30623790b0ecd,R503929753,R503929753,2023-01-04 19:00:58.000,1a22bb418515935568e30623790b0ecd
4,2023-01-04,1a22bb418515935568e30623790b0ecd,R503929753,R503929753,2023-01-04 19:00:58.000,1a22bb418515935568e30623790b0ecd
5,2023-01-29,8b9eaf2aa252948f781f3f397c7c3e07,C00448915,C00448915,2023-01-29 05:06:17.000,8b9eaf2aa252948f781f3f397c7c3e07
6,2023-02-28,7601e0962d704e0c46472fd4fba19fba,R327164481,R327164481,2023-02-28 22:46:53.476,7601e0962d704e0c46472fd4fba19fba
7,2023-02-28,7601e0962d704e0c46472fd4fba19fba,R908685518,R908685518,2023-02-28 23:39:46.188,7601e0962d704e0c46472fd4fba19fba
8,2023-02-28,7601e0962d704e0c46472fd4fba19fba,R327164481,R327164481,2023-02-28 22:46:53.476,7601e0962d704e0c46472fd4fba19fba
9,2023-03-15,0b2f470d25b19c763c09e7417e99911a,R456672697,R456672697,2023-03-15 20:05:05.433,0b2f470d25b19c763c09e7417e99911a


In [25]:
# checking shape
ctr_customers_merged.shape

(268, 6)

In [26]:
# checking shape
trt_customers_merged.shape

(10383, 6)

In [27]:
# viewing null order numbers 
trt_customers_merged[trt_customers_merged['order_number'].isna()]


,ConversionDate,Md5CUS,OrderNumberCUS,order_number,order_completed_at,email
542,2023-03-10,98a55cbcd6f404e9d5fb734a22784d0f,R617678330,NaN,NaT,NaN
817,2023-03-12,1106420fea66bad1a60b76fc84a09b45,C00468912,NaN,NaT,NaN
819,2023-03-12,1106420fea66bad1a60b76fc84a09b45,C00468912,NaN,NaT,NaN
1075,2023-03-05,adbc1447b21bccbd0be9cf99b5bf782f,C00466320,NaN,NaT,NaN


In [28]:
# exporting 
trt_customers_merged.to_csv('TRT_conversion_matched.csv')

# CTR Group File

In [2]:
# reading control group
control_group_df = pd.read_csv('KineticCasperMattress_Q1_2023_CTR 2.csv',sep='|',index_col=False, encoding="ISO-8859-1")
control_group_df.head()

,MD5,Exec_date
0,00000ac959de1c7864daa54d10a62fbd,2023-02-14
1,00003552ee538065ed03dc27082a43ac,2023-02-13
2,00003651d7bb4f0112cdb5597cbbdcf0,2023-02-15
3,00003c4f2bf990815ee9f0376cbbd25a,2023-02-15
4,0000571828826ffbd8195947736d45fc,2023-02-16


In [3]:
# number of rows in the control group
len(control_group_df["MD5"])

13322055

In [10]:
#shape of the dataframe 
control_group_df.shape

(13322055, 2)

In [11]:
#merging zeta file and database query 
control_merged = control_group_df.merge(casperdb_customers, left_on = ['MD5'], right_on = ['email'], how='left')


In [12]:
# remove any rows where DB is null
control_nulls_removed = control_merged.dropna()

In [13]:
#shape of file after removing null values
control_nulls_removed.shape

(433, 5)

In [14]:
# exporting
control_nulls_removed.to_csv('CRT_conversion_matched.csv')

# TRT FILES

In [12]:
# ADDING all file names in a list
file_names = ['KineticCasperMattress_Q1_2023_TRT_01.csv',
         'KineticCasperMattress_Q1_2023_TRT_02.csv',
         'KineticCasperMattress_Q1_2023_TRT_03.csv',
         'KineticCasperMattress_Q1_2023_TRT_04.csv',
         'KineticCasperMattress_Q1_2023_TRT_05.csv',
         'KineticCasperMattress_Q1_2023_TRT_06.csv',
         'KineticCasperMattress_Q1_2023_TRT_07.csv',
         'KineticCasperMattress_Q1_2023_TRT_08.csv',
         'KineticCasperMattress_Q1_2023_TRT_09.csv',
         'KineticCasperMattress_Q1_2023_TRT_10.csv',
         'KineticCasperMattress_Q1_2023_TRT_11.csv',
         'KineticCasperMattress_Q1_2023_TRT_12.csv',
         'KineticCasperMattress_Q1_2023_TRT_13.csv']

In [28]:
# reading each file, dropping nulls, and adding to a final dataframe 
df_trt_final = []

for x in file_names:
    
    df = pd.read_csv(x ,sep='|',index_col=False, encoding="ISO-8859-1")
    df_merged = df.merge(casperdb_customers, left_on = ['MD5'], right_on = ['email'], how='left')
    df_na_removed = df_merged.dropna()
    df_trt_final.append(df_na_removed)
    
df_trt_final = pd.concat(df_trt_final)

In [29]:
# checking shape of final df
df_trt_final.shape

(13669, 7)

In [30]:
# exporting 
df_trt_final.to_csv('TRT_conversion_matched.csv')

## QA

In [16]:

total = 0
for x in file_names:
    df = pd.read_csv(x ,sep='|',index_col=False, encoding="ISO-8859-1")
    length = len(df['MD5'])
    total = total + length

print(total)

119953679


In [3]:
df2 = pd.read_csv('trt_dup.csv')

In [4]:
df2.head()

,email,order_number,Exec_date,order_completed_at
0,0003d9500c812183b1b0637fc8f18f20,C00467795,3/8/23,3/9/23 22:03
1,0026fec261606c20e495b147a72f0985,C00451747,2/3/23,2/4/23 13:58
2,002959a0f4de518d5fd8bd2ed4651f78,R543364835,1/18/23,1/18/23 20:13
3,003f2ba8f3edb719ba8df41278a63a1e,R178330177,2/2/23,2/5/23 20:32
4,004494d850e9b40f7090aed092f4dd30,C00464705,3/1/23,3/1/23 0:19


In [5]:
df2.dtypes

email                 object
order_number          object
Exec_date             object
order_completed_at    object
dtype: object

In [8]:
df2.shape

(9422, 4)

In [6]:
df_dropped = df2.drop_duplicates(subset=["email", "order_number"], keep='first')


In [9]:
df_dropped.shape

(7449, 4)

In [10]:
df_dropped.to_csv('trt_dup_removed.csv')

In [11]:
df3 = pd.read_csv('trt_dup_removed_2.csv')

In [12]:
df3.head()

,email,order_number,Exec_date,order_completed_at
0,0003d9500c812183b1b0637fc8f18f20,C00467795,3/8/23,3/9/23 22:03
1,0026fec261606c20e495b147a72f0985,C00451747,2/3/23,2/4/23 13:58
2,002959a0f4de518d5fd8bd2ed4651f78,R543364835,1/18/23,1/18/23 20:13
3,003f2ba8f3edb719ba8df41278a63a1e,R178330177,2/2/23,2/5/23 20:32
4,004494d850e9b40f7090aed092f4dd30,C00464705,3/1/23,3/1/23 0:19


In [13]:
df2_dropped = df3.drop_duplicates(subset=["email"], keep='first')


In [14]:
df2_dropped.shape

(6841, 4)

In [15]:
df2_dropped.to_csv('trt_dup_removed_new.csv')